In [ ]:
!pip install numpy==1.26.4 matplotlib==3.9.2 mpmath==1.3.0 scipy==1.14.1 plotly==5.24.1 sympy==1.13.3

In [ ]:
import numpy as np
import plotly.graph_objects as go
from mpmath import mp, chi
from sympy import zeta as sympy_zeta
from functools import lru_cache
from scipy.interpolate import interp1d

# Настройки mpmath
mp.dps = 30

# Параметры
T = 50
t = np.linspace(-T, T, 300)
gamma_n = [14.1347, 21.0220, 25.0108]
k = 1.0
epsilon = 1e-8
omega = 1e12
hbar = 6.582e-16
dt_avg = 2 * T / len(t)

# Кэширование zeta и chi
@lru_cache(maxsize=1000)
def cached_zeta(s_real, s_imag):
    try:
        return complex(sympy_zeta(complex(s_real, s_imag)))
    except:
        return 0.0

@lru_cache(maxsize=1000)
def cached_chi(s_real, s_imag):
    try:
        return complex(chi(complex(s_real, s_imag)))
    except:
        return 0.0

# Модуль |zeta(1/2 + it)|^2
def zeta_squared(u):
    z = cached_zeta(0.5, u)
    result = z * z.conjugate()
    return float(result.real) if not np.isnan(result) and not np.isinf(result) else 0.0

# Функция psi
def psi(s, u, epsilon):
    denom = (s - 0.5 - 1j*u)**2 * (1 - s - 0.5 - 1j*u)**2 + epsilon**2
    result = 1 / denom
    zeta_s = cached_zeta(s.real, s.imag)
    chi_s = cached_chi(s.real, s.imag)
    chi_zeta = abs(chi_s * zeta_s)**2
    if chi_zeta > 1e10:
        chi_zeta = 1e10
    result *= chi_zeta
    return result if not np.isnan(result) and not np.isinf(result) else 0.0

# Вычисление ядра K_sym
def compute_kernel(epsilon):
    N = len(t)
    K = np.zeros((N, N), dtype=complex)
    for i in range(N):
        for j in range(N):
            s = 0.5 + 1j * t[i]
            u = t[j]
            psi_val = psi(s, u, epsilon)
            zeta_val = zeta_squared(u)
            K[i, j] = psi_val * zeta_val
    K = (K + K.conj().T) / 2
    return K

# Потенциал
V_eff = -k * np.array([zeta_squared(ti) for ti in t])

# Вычисление собственных функций
K_sym = compute_kernel(epsilon) * dt_avg
eigvals, eigvecs = np.linalg.eigh(K_sym)
idx = np.argsort(eigvals)[::-1]
eigvals = eigvals[idx]
eigvecs = eigvecs[:, idx]

# Выбор трёх собственных функций
selected_indices = [np.argmin(np.abs(eigvals - gamma)) for gamma in gamma_n]
fn_data = [np.abs(eigvecs[:, idx])**2 for idx in selected_indices]

# Интерполяция
fn_interps = [interp1d(t, fn, kind='cubic') for fn in fn_data]

# Плотности вероятности с фазой
psi_data = []
for i, gamma in enumerate(gamma_n):
    frames = []
    for time in np.linspace(0, 1e-12, 20):
        phase = omega * gamma * time
        psi = fn_interps[i](t) * np.exp(1j * phase)
        psi_squared = np.abs(psi)**2
        frames.append(psi_squared)
    psi_data.append(frames)

# Описание в виде цитаты
print("""
> * Любо́н (Lub): Описание и цвета графика *
> ----------------------------------------
> @ Тип: Скалярный бозон (спин-0)
> # Масса: ~10^-3 эВ/c^2 (легче нейтрино)
> ~ Энергия: E_n = ℏωγ_n, где γ_n — нули ζ(1/2 + iγ_n)
> & Природа: Резонанс квантовой системы с хаотической динамикой
> = Связь: Дзета-функция Римана ζ(s)
> ? Где искать: Квантовые точки, фотонные кристаллы, космос
> % Тёмная материя?: Возможный кандидат
> $ Статистика: GUE (хаотический спектр)
> ! Значение: Мост между математикой и физикой
>
> + Цвета линий на графике:
> - # Синий: |lub_1(t)|^2 (γ_1 = 14.1347)
> - # Зелёный: |lub_2(t)|^2 (γ_2 = 21.0220)
> - # Фиолетовый: |lub_3(t)|^2 (γ_3 = 25.0108)
> - # Красный (пунктир): Потенциал V_eff(t) = -|ζ(1/2 + it)|^2
> - # Полупрозрачная красная поверхность: Потенциал V_eff(t)
> ----------------------------------------
""")

# Создание графика
fig = go.Figure()

# Волновые функции
colors = ['blue', 'green', 'purple']
for i, gamma in enumerate(gamma_n):
    fig.add_trace(go.Scatter3d(
        x=t,
        y=psi_data[i][0],
        z=V_eff,
        mode='lines',
        line=dict(width=5, color=colors[i]),
        name=f'|lub_{i+1}(t)|^2 (γ_{i+1} = {gamma})',
        visible=(i == 0)
    ))

# Потенциал
fig.add_trace(go.Scatter3d(
    x=t,
    y=np.zeros_like(t),
    z=V_eff,
    mode='lines',
    line=dict(width=4, color='red', dash='dash'),
    name='V_eff(t)'
))

# Поверхностный график
X, Y = np.meshgrid(t, np.linspace(0, max(psi_data[0][0]) * 1.1, 20))
Z = np.tile(V_eff, (20, 1))
fig.add_trace(go.Surface(
    x=X,
    y=Y,
    z=Z,
    opacity=0.3,
    colorscale='Reds',
    showscale=False,
    name='Потенциал (поверхность)'
))

# Аннотации
for gamma in gamma_n:
    fig.add_trace(go.Scatter3d(
        x=[gamma],
        y=[max(psi_data[0][0]) * 0.5],
        z=[min(V_eff)],
        mode='text',
        text=[f'γ_{gamma}'],
        textposition='top center',
        showlegend=False
    ))

# Анимация
frames = []
for f in range(20):
    frame_data = []
    for i, gamma in enumerate(gamma_n):
        frame_data.append(go.Scatter3d(
            x=t,
            y=psi_data[i][f],
            z=V_eff,
            mode='lines',
            line=dict(width=5, color=colors[i]),
            name=f'|lub_{i+1}(t)|^2',
            visible=(i == 0)
        ))
    frame_data.append(fig.data[len(gamma_n)])
    frame_data.append(fig.data[len(gamma_n) + 1])
    for j in range(len(gamma_n)):
        frame_data.append(fig.data[len(gamma_n) + 2 + j])
    frames.append(go.Frame(data=frame_data, name=f'frame{f}'))

fig.frames = frames

# Слайдеры
sliders = [
    dict(
        steps=[
            dict(
                method='update',
                args=[{'visible': [k == i for k in range(len(gamma_n))] + [True, True] + [True]*len(gamma_n)},
                      {'title': f'Любо́н: |lub_{i+1}(t)|^2 (γ_{i+1} = {gamma_n[i]})'}],
                label=f'γ_{i+1}'
            ) for i in range(len(gamma_n))
        ],
        active=0,
        currentvalue={'prefix': 'Выбор нуля: '},
        pad={'t': 50}
    )
]

# Кнопки анимации
fig.update_layout(
    updatemenus=[
        dict(
            type='buttons',
            showactive=True,
            buttons=[
                dict(
                    label='Играть',
                    method='animate',
                    args=[None, {'frame': {'duration': 50, 'redraw': True}, 'fromcurrent': True}]
                ),
                dict(
                    label='Пауза',
                    method='animate',
                    args=[[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate'}]
                )
            ],
            pad={'r': 10, 't': 10}
        )
    ],
    sliders=sliders,
    title='Анимированная 3D-визуализация Любо́на (Lub)',
    scene=dict(
        xaxis_title='t',
        yaxis_title='|lub(t)|^2',
        zaxis_title='V_eff(t)',
        xaxis=dict(range=[-T, T]),
        yaxis=dict(range=[0, max(psi_data[0][0]) * 1.1]),
        zaxis=dict(range=[min(V_eff) * 1.1, max(V_eff) * 1.1])
    ),
    showlegend=True,
    width=800,
    height=600,
    margin=dict(l=0, r=0, t=50, b=50),
    scene_aspectmode='manual',
    scene_aspectratio=dict(x=1, y=1, z=0.5),
    template='plotly_white',
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    autosize=False,
    scene_camera=dict(eye=dict(x=1.5, y=1.5, z=0.8))
)

# Показать график
fig.show()

# Экспорт в HTML
fig.write_html("lubon_visualization.html")
print("* График сохранён как lubon_visualization.html")